In [1]:
import requests
import hmac
import json
import base64
import hashlib
import pandas as pd
import datetime, time
from time import sleep

In [2]:
base_url = "https://api.sandbox.gemini.com"
endpoint = "/v1/order/new"
url = base_url + endpoint

In [3]:
gemini_api_key = "master-2rJeGEcrQbx6OKXvSvLR"
gemini_api_secret = "CZTk6cmFZgVPSnoHij22LZNFZ3S".encode()

In [5]:
while True:
    t = datetime.datetime.now()
    payload_nonce =  str(int(time.mktime(t.timetuple())*1000))
    try:
        base_ticker_url = "https://api.gemini.com/v1"
        response = requests.get(base_ticker_url + "/pubticker/btcusd")
        btc_data = response.json()
        btc_price = float(btc_data['ask'])
    except exception as e:
        print('Error obtaining data')
        
    if btc_price >= 37900.00:
        try:
            response = requests.get(base_ticker_url + "/pubticker/ethusd")
            eth_data = response.json()
            eth_price = float(eth_data['ask'])
        except exception as e:
            print('Error obtaining data')
            
        payload = {
            "request": "/v1/order/new",
            "nonce": payload_nonce,
            "symbol": "ethusd",
            "amount": "1",
            "price": eth_price,
            "side": "buy",
            "type": "exchange limit",
            "account": "algotrading"
            }
        
        encoded_payload = json.dumps(payload).encode()
        b64 = base64.b64encode(encoded_payload)
        signature = hmac.new(gemini_api_secret, b64, hashlib.sha384).hexdigest()
        
        request_headers = { 'Content-Type': "text/plain",
                    'Content-Length': "0",
                    'X-GEMINI-APIKEY': gemini_api_key,
                    'X-GEMINI-PAYLOAD': b64,
                    'X-GEMINI-SIGNATURE': signature,
                    'Cache-Control': "no-cache" }
        
        try:
            response = requests.post(url,
                            data=None,
                            headers=request_headers)
            
            new_order = response.json()
            print(new_order)
            check_status = new_order['client_order_id']
            
            sleep(2)
            
            t = datetime.datetime.now()
            nonce =  str(int(time.mktime(t.timetuple())*1000))
            
            status = requests.get('https://api.gemini.com/v1/order/status/'+ nonce+'/'+ check_status)
            
            if status['is_true'] == 'True':
                print(status)
                print ('Order placed at {}'.format(pd.Timestamp.now()))
                break
            
            else:
                print('Order was canceled {}'.format(pd.Timestamp.now()))
                break
            
        except Exception as e:
            print('Error placing order')
    else:
        print('BTC did not reach the specified price')
        sleep(10)
        continue
        
        

{'order_id': '716494276', 'id': '716494276', 'symbol': 'ethusd', 'exchange': 'gemini', 'avg_execution_price': '1193.78', 'side': 'buy', 'type': 'exchange limit', 'timestamp': '1610114431', 'timestampms': 1610114431202, 'is_live': False, 'is_cancelled': False, 'is_hidden': False, 'was_forced': False, 'executed_amount': '1', 'options': [], 'price': '1255.20', 'original_amount': '1', 'remaining_amount': '0'}
Error placing order
{'order_id': '716494354', 'id': '716494354', 'symbol': 'ethusd', 'exchange': 'gemini', 'avg_execution_price': '1193.78', 'side': 'buy', 'type': 'exchange limit', 'timestamp': '1610114432', 'timestampms': 1610114432784, 'is_live': False, 'is_cancelled': False, 'is_hidden': False, 'was_forced': False, 'executed_amount': '1', 'options': [], 'price': '1254.57', 'original_amount': '1', 'remaining_amount': '0'}
Error placing order
{'order_id': '716494443', 'id': '716494443', 'symbol': 'ethusd', 'exchange': 'gemini', 'avg_execution_price': '1193.78', 'side': 'buy', 'type'

NameError: name 'exception' is not defined